<a href="https://colab.research.google.com/github/ombuijabali/Coconut_Pedcition_Model/blob/main/Model_with_multiple_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install package

In [2]:
!pip install geoai-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 5.0 MB/s eta 0:00:00
  

#Import libraries

In [1]:
import geoai

# List of training raster and vector files (Add more as needed)

In [2]:

train_raster_urls = [
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_train.tif",
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/cars_7cm.tif",
]
train_vector_urls = [
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_train_buildings.geojson",
    "https://huggingface.co/datasets/construmgis/geospatial/resolve/main/HY.geojson"]

test_raster_url = (
    "https://huggingface.co/datasets/giswqs/geospatial/resolve/main/naip_test.tif"
)

# Download all files

In [3]:
train_raster_paths = [geoai.download_file(url) for url in train_raster_urls]
train_vector_paths = [geoai.download_file(url) for url in train_vector_urls]
test_raster_path = geoai.download_file(test_raster_url)

naip_train.tif: 100%|██████████| 12.1M/12.1M [00:00<00:00, 60.4MB/s]
cars_7cm.tif: 100%|██████████| 92.0M/92.0M [00:02<00:00, 37.4MB/s]
naip_train_buildings.geojson: 100%|██████████| 456k/456k [00:00<00:00, 5.08MB/s]
HY.geojson: 100%|██████████| 5.13k/5.13k [00:00<00:00, 12.2MB/s]
naip_test.tif: 100%|██████████| 19.7M/19.7M [00:00<00:00, 32.3MB/s]


# View first vector over a raster (optional)

In [4]:
geoai.view_vector_interactive(train_vector_paths[0], tiles=train_raster_urls[0])
geoai.view_raster(test_raster_url)

# Folder to store output tiles

In [5]:
out_folder = "output"

# Export tiles from multiple raster/vector pairs

In [6]:
for train_raster, train_vector in zip(train_raster_paths, train_vector_paths):
    geoai.export_geotiff_tiles(
        in_raster=train_raster,
        out_folder=out_folder,
        in_class_data=train_vector,
        tile_size=512,
        stride=256,
        buffer_radius=0,
    )


Raster info for naip_train.tif:
  CRS: EPSG:26911
  Dimensions: 2503 x 1126
  Resolution: (0.5999999999999953, 0.5999999999996691)
  Bands: 4
  Bounds: BoundingBox(left=454780.8, bottom=5277567.0, right=456282.6, top=5278242.6)
Loaded 722 features from naip_train_buildings.geojson
Vector CRS: EPSG:4326
Reprojecting features from EPSG:4326 to EPSG:26911
Found 6 unique classes: ['apartments' None 'terrace' 'detached' 'house' 'shed']


Generated: 36, With features: 36: 100%|██████████| 36/36 [00:22<00:00,  1.61it/s]



------- Export Summary -------
Total tiles exported: 36
Tiles with features: 36 (100.0%)
Average feature pixels per tile: 45966.3
Output saved to: output

------- Georeference Verification -------

Raster info for cars_7cm.tif:
  CRS: EPSG:3857
  Dimensions: 8351 x 4463
  Resolution: (0.07464844311342499, 0.07464346097829833)
  Bands: 3
  Bounds: BoundingBox(left=-10622651.27699905, bottom=3462200.907507864, right=-10622027.88785061, top=3462534.04127421)
Loaded 17 features from HY.geojson
Vector CRS: EPSG:4326
Reprojecting features from EPSG:4326 to EPSG:3857


Generated: 544, With features: 262: 100%|██████████| 544/544 [00:32<00:00, 16.57it/s]


------- Export Summary -------
Total tiles exported: 544
Tiles with features: 262 (48.2%)
Average feature pixels per tile: 83563.1
Output saved to: output

------- Georeference Verification -------


# Train the model with multiple images

In [ ]:
geoai.train_MaskRCNN_model(
    images_dir=f"{out_folder}/images",
    labels_dir=f"{out_folder}/labels",
    output_dir=f"{out_folder}/models",
    num_channels=3,
    pretrained=True,
    batch_size=4,
    num_epochs=10,
    learning_rate=0.005,
    val_split=0.2,
)

Using device: cpu
Found 544 image files and 544 label files
Training on 435 images, validating on 109 images


Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:01<00:00, 140MB/s]


Epoch: 0, Batch: 0/109, Loss: 1.7445, Time: 79.10s
Epoch: 0, Batch: 10/109, Loss: 27.1487, Time: 735.66s


# Object detection on test raster

In [ ]:
masks_path = "naip_test_prediction.tif"
model_path = f"{out_folder}/models/best_model.pth"

geoai.object_detection(
    test_raster_path,
    masks_path,
    model_path,
    window_size=512,
    overlap=256,
    confidence_threshold=0.5,
    batch_size=4,
    num_channels=3,
)

# Convert predicted masks to GeoJSON

In [ ]:
output_path = "naip_test_prediction.geojson"
gdf = geoai.orthogonalize(masks_path, output_path, epsilon=2)

# View results

In [ ]:
geoai.view_vector_interactive(output_path, tiles=test_raster_url)

# Create a split map for comparison

In [ ]:
geoai.create_split_map(
    left_layer=output_path,
    right_layer=test_raster_url,
    left_args={"style": {"color": "red", "fillOpacity": 0.2}},
    basemap=test_raster_url,
)